<h1>Model Inference with RAG Implementation</h1>

<h1>Model Inference</h1>

<h3>using: /v1/completions endpoint</h3>
<h4>*More for Single-shot prompts, single-turn Q&A or document completion</h4>

In [9]:
# Installing required libraries
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 363.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 341.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 400.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 650.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [10]:
#Import required libraries
import requests
import pandas as pd
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch, helpers
from tqdm import tqdm
from elasticsearch.helpers import bulk

In [12]:
infer_endpoint = "http://model-predictor.minio.svc.cluster.local:8080" #Change infer endpoint here
infer_url = f"{infer_endpoint}/v1/completions"

In [13]:
# get the model name 
# for testing only
!curl http://model-predictor.minio.svc.cluster.local:8080/v1/models

{"object":"list","data":[{"id":"model","object":"model","created":1745841436,"owned_by":"vllm","root":"/mnt/models","parent":null,"max_model_len":4096,"permission":[{"id":"modelperm-ca5dee3fcbe5471da4109e7dc0a168ea","object":"model_permission","created":1745841436,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [14]:
# Test for hyperparameter tuning 
payload = {
    "model": "model",
    "prompt": "What's the difference between OpenShift and Kubernetes?",
    "max_tokens": 1024,
    "temperature": 0.3, # Controls randomness, lower temp for factuality 
    "top_p": 1,
    "n": 1, # Number of completions to generate
    "repetition_penalty": 1.1, # Penalize repeated tokens (1 = no penalty)
    "presence_penalty": 0.2, # Discourage mentioning same concepts again
    "frequency_penalty": 0.2, # Discourage repeating the *same words* too frequently
    "stream": False # If True, stream tokens back (like a live typewriter)
}

response = requests.post(infer_url, json=payload)

# prints the whole response json
# print(response.json())

output_body = response.json()
generated_response = output_body['choices'][0]['text']
print(generated_response.strip())

OpenShift is a container application platform built around a core of Docker container packaging and Kubernetes orchestration. It adds developer and operations-centric tools on top of Kubernetes to enable rapid application development, multi-cloud deployments, and portability across cloud providers. In other words, OpenShift is a distribution of Kubernetes that includes additional features and tools for developers and operations teams.


In [15]:
# Test for hyperparameter tuning 
payload = {
    "model": "model",
    "prompt": "What's the difference between OpenShift and Kubernetes?",
    "max_tokens": 200,
    "temperature": 0.7
}

response = requests.post(infer_url, json=payload)

# prints the whole response json
# print(response.json())

output_body = response.json()
generated_response = output_body['choices'][0]['text']
print(generated_response.strip())

OpenShift and Kubernetes are both open-source container orchestration systems, but they have some key differences:

1. **Origin and Development**: Kubernetes was originally developed by Google and is now maintained by the Cloud Native Computing Foundation (CNCF). OpenShift, on the other hand, was developed by Red Hat and is based on Kubernetes.

2. **Ease of Use**: OpenShift is designed to be more user-friendly and easier to set up, especially for developers who are not Kubernetes experts. It provides a web console, CLI tools, and integrated development environments (IDEs) for managing applications. Kubernetes, while powerful, can be more complex to set up and manage, requiring a deeper understanding of its components and APIs.

3. **Built-in Features**: OpenShift comes with several built-in features that are not part of the standard Kubernetes distribution. These include:
   - **Source-to-Image (S2I)**: A


<h1>Model Inference</h1>

<h3>using: /v1/chat/completions endpoint</h3>
<h3>*More for chat-tuned, multi-turn conversations, model behaving like assistant</h3>

In [7]:
##TO DO into the Prompt Template
# preserve messages history 
# keep older messages if they are essential for content 
# truncate long histories by summarizing/dropping older exchanges 

# Prompt token - is there a max? 

# create a Python class to connect to FE 

In [33]:
infer_endpoint = "http://model-predictor.minio.svc.cluster.local:8080" #Change infer endpoint here
infer_url = f"{infer_endpoint}/v1/chat/completions"

In [34]:
# get the model name 
# for testing only
!curl http://model-predictor.minio.svc.cluster.local:8080/v1/models

{"object":"list","data":[{"id":"model","object":"model","created":1745841624,"owned_by":"vllm","root":"/mnt/models","parent":null,"max_model_len":4096,"permission":[{"id":"modelperm-46acfada92ca428e9477a3c99a8587ec","object":"model_permission","created":1745841624,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [35]:
user_question_one = "What's the difference between OpenShift & Kubernetes?"

messages = [
    {"role": "system", "content": "You are a chat bot assistant for helping people with their queries."},
    {"role": "user", "content": user_question_one}
]


payload = {
    "model": "model",
    "messages": messages,
    "max_tokens": 1024,
    "temperature": 0.3, # Controls randomness, lower temp for factuality 
    "top_p": 1,
    "n": 1, # Number of completions to generate
    "repetition_penalty": 1.1, # Penalize repeated tokens (1 = no penalty)
    "presence_penalty": 0.2, # Discourage mentioning same concepts again
    "frequency_penalty": 0.2, # Discourage repeating the *same words* too frequently
    "stream": False # If True, stream tokens back (like a live typewriter)
}

response = requests.post(infer_url, json=payload)

# prints the whole response json
# print(response.json())

output_body = response.json()
generated_response = output_body['choices'][0]['message']['content']
print(generated_response.strip())

# add to the messages list 
messages.append({"role": "system", "content": generated_response.strip()})

OpenShift and Kubernetes are both containerization platforms, but they have some key differences:

1. **Origin and Ownership**: Kubernetes (K8s) is an open-source platform developed by Google based on their internal system Borg. It's managed by the Cloud Native Computing Foundation (CNCF). On the other hand, OpenShift is a commercial product from Red Hat (now part of IBM), built on top of Kubernetes.

2. **Ease of Use**: OpenShift is often considered more user-friendly than Kubernetes because it offers a more streamlined experience out of the box. It includes features like automated setup, integrated development environment tools, and simplified application deployment processes. 

3. **Additional Features**: OpenShift comes with additional features that aren't included in standard Kubernetes, such as a web console for managing clusters, built-in continuous integration/continuous delivery (CI/CD) pipelines, and integrated image registry. 

4. **Support and Documentation**: Being a comme

In [11]:
# ongoing conversations 

user_question_two = "I'm interested in implementing AI with RAG approach, tell me more about how i can do it with elastic as the vector database"

messages.append({"role": "user", "content": user_question_two})

payload = {
    "model": "model",
    "messages": messages,
    "max_tokens": 1024,
    "temperature": 0.3, # Controls randomness, lower temp for factuality 
    "top_p": 1,
    "n": 1, # Number of completions to generate
    "repetition_penalty": 1.1, # Penalize repeated tokens (1 = no penalty)
    "presence_penalty": 0.2, # Discourage mentioning same concepts again
    "frequency_penalty": 0.2, # Discourage repeating the *same words* too frequently
    "stream": False # If True, stream tokens back (like a live typewriter)
}

response = requests.post(infer_url, json=payload)

# prints the whole response json
# print(response.json())

output_body = response.json()
generated_response = output_body['choices'][0]['message']['content']
print(generated_response.strip())

Implementing AI with Retrieval-Augmented Generation (RAG) using Elasticsearch as the vector database involves several steps. Here's a simplified guide to help you get started:

1. **Data Preparation**: Firstly, you need to prepare your data. Your dataset should consist of text passages that will serve as your knowledge base. These texts will be converted into vectors (numerical representations) that can be stored and queried in Elasticsearch.

2. **Vectorization**: Convert your text data into numerical vectors using techniques like Word2Vec, GloVe, or Sentence Transformers' universal sentence encoder. These methods convert words or sentences into high-dimensional vectors where semantically similar items are close together in the vector space.

3. **Indexing Vectors in Elasticsearch**: After generating vectors, you'll need to index them in Elasticsearch. You can store these vectors directly in Elasticsearch documents alongside their corresponding textual representation. To facilitate fa

<h1>Create Embeddings in Elastic Vector Database</h1>

In [16]:
df = pd.read_csv("cleaned_helpdesk_data.csv", low_memory=False)
print(df.head())

print('\n')

print(df.columns.tolist())

   issue_id  answer_id author              creation_time  \
0    456983          0  almet  2010-12-05T17:31:55+00:00   
1    456983          1  Gui13  2010-12-05T17:43:43+00:00   
2    456983          2  almet  2010-12-05T17:45:28+00:00   
3    456983          3  almet  2010-12-14T15:57:04+00:00   
4    456983          4  Gui13  2010-12-21T21:02:19+00:00   

                                          issue_body  \
0  Hey, Say I add an image in my article, giving ...   
1  Hey, Say I add an image in my article, giving ...   
2  Hey, Say I add an image in my article, giving ...   
3  Hey, Say I add an image in my article, giving ...   
4  Hey, Say I add an image in my article, giving ...   

                                         answer_body  
0  If we do add the SITEURL value, it can break b...  
1  A bit hackish for sure, but I'm not yet good e...  
2  okay, let's go for that so. Will do that when ...  
3  arnaud is currently working on that, see his f...  
4                     Shoul

In [17]:
# testing - create a smaller csv
df = pd.read_csv("cleaned_helpdesk_data.csv")
# df_sample = df.head(100)
# df_sample.to_csv("helpdesk_small_sample.csv", index=False)

In [18]:
!curl -u elastic:1rdzVXU8i8F9T8hs1p6c78d6 -k https://elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com

{
  "name" : "elasticsearch-sample-es-default-0",
  "cluster_name" : "elasticsearch-sample",
  "cluster_uuid" : "nr3Sh4ArQcqFm90rHd5Smw",
  "version" : {
    "number" : "8.17.0",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "2b6a7fed44faa321997703718f07ee0420804b41",
    "build_date" : "2024-12-11T12:08:05.663969764Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [19]:
%pip install elasticsearch==8.10.0


[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
!pip show elasticsearch

Name: elasticsearch
Version: 8.10.0
Summary: Python client for Elasticsearch
Home-page: https://github.com/elastic/elasticsearch-py
Author: Elastic Client Library Maintainers
Author-email: client-libs@elastic.co
License: Apache-2.0
Location: /opt/app-root/lib64/python3.11/site-packages
Requires: elastic-transport
Required-by: 


In [21]:
from elasticsearch import Elasticsearch

In [18]:
# elasticsearch client 
import requests
from requests.auth import HTTPBasicAuth

url = "https://elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com"
auth = HTTPBasicAuth("elastic", "1rdzVXU8i8F9T8hs1p6c78d6")

response = requests.get(url, auth=auth, verify=False)

print(response.status_code)
print(response.json())


200
{'name': 'elasticsearch-sample-es-default-1', 'cluster_name': 'elasticsearch-sample', 'cluster_uuid': 'nr3Sh4ArQcqFm90rHd5Smw', 'version': {'number': '8.17.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b6a7fed44faa321997703718f07ee0420804b41', 'build_date': '2024-12-11T12:08:05.663969764Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
# from elasticsearch import Elasticsearch

es = Elasticsearch(
    hosts=["https://elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com"],
    basic_auth=('elastic', '1rdzVXU8i8F9T8hs1p6c78d6'),
    verify_certs=False
)

print(es.info())


{'name': 'elasticsearch-sample-es-default-1', 'cluster_name': 'elasticsearch-sample', 'cluster_uuid': 'nr3Sh4ArQcqFm90rHd5Smw', 'version': {'number': '8.17.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b6a7fed44faa321997703718f07ee0420804b41', 'build_date': '2024-12-11T12:08:05.663969764Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


/opt/app-root/lib64/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:397: SecurityWarning: Connecting to 'https://elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [134]:
# LOGIC
# ideally, all the contents of 'cleaned_body' field from the continous answer_ids grouped under the same issue_id should be in a single vector embedding 

# Load your job dataset
# df = pd.read_csv("helpdesk_small_sample.csv")

# not all values in clean_body are strings, thus throwing TypeError if this line is not runned 
df["answer_body"] = df["answer_body"].fillna("").astype(str)

# Group answers by issue_id and concatenate them in order
grouped = (
    df.sort_values(by=["issue_id", "answer_id"])
      .groupby("issue_id")
      .agg({
          "issue_body": "first",
          "answer_body": lambda x: " ".join(x)
      })
      .reset_index()
)

# Combine issue_body and answer_body into a single string for embedding
grouped["full_text"] = grouped["issue_body"] + " " + grouped["answer_body"]

# Generate embeddings
grouped["embedding"] = model.encode(grouped["full_text"]).tolist()

# grouped = df.sort_values(by=["issue_id", "answer_id"]).groupby("issue_id")["clean_body"].apply(lambda texts: " ".join(texts)).reset_index()


# Embedding Model 
# model = SentenceTransformer("all-MiniLM-L6-v2")  # faced out of memory error 
# model = SentenceTransformer("paraphrase-MiniLM-L3-v2", device="cuda")  # lighter version
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device='cpu')


# elasticsearch client 
es = Elasticsearch(
    hosts=["https://elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com"], 
    basic_auth=('elastic', '1rdzVXU8i8F9T8hs1p6c78d6'), 
    verify_certs=False
)
print(es.info())

index_name = "helpdesk-embeddings"
if not es.indices.exists(index=index_name):
    es.indices.create(
        index=index_name,
        body={
            "mappings": {
                "properties": {
                    "issue_id": {"type": "keyword"},
                    "issue_body": {"type": "text"},
                    "answer_body": {"type": "text"},
                    "embedding": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    }
                }
            }
        }
    )

# Prepare docs for bulk indexing
def create_docs(data):
    for _, row in data.iterrows():
        yield {
            "_index": index_name,
            "_id": row["issue_id"],
            "_source": {
                "issue_id": row["issue_id"],
                "issue_body": row["issue_body"],
                "answer_body": row["answer_body"],
                "embedding": row["embedding"]
            }
        }

# Bulk upload
bulk(es, create_docs(grouped))
print("✅ Embeddings stored successfully in Elasticsearch.")

/opt/app-root/lib64/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:403: SecurityWarning: Connecting to 'https://elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(


{'name': 'elasticsearch-sample-es-default-1', 'cluster_name': 'elasticsearch-sample', 'cluster_uuid': 'nr3Sh4ArQcqFm90rHd5Smw', 'version': {'number': '8.17.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b6a7fed44faa321997703718f07ee0420804b41', 'build_date': '2024-12-11T12:08:05.663969764Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
✅ Embeddings stored successfully in Elasticsearch.


In [26]:
## for debugging/testing purposes only 

# elasticsearch client 
es = Elasticsearch(
    hosts=["https://elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com"], 
    basic_auth=('elastic', '1rdzVXU8i8F9T8hs1p6c78d6'), 
    verify_certs=False
)

# view available embedding indices 
indices = es.indices.get_alias().keys()
print("Available indices:", list(indices))

# view all stored embeddings 
results = es.search(
    index="helpdesk-embeddings",
    size=10,  # adjust as needed
    _source=["embedding"]  # only return embedding field
)

# view the number of documents in the stored index 
count = es.count(index="helpdesk-embeddings")["count"]
print(f"Num of embeddings in index: {count}")


for doc in results["hits"]["hits"]:
    print(f"ID: {doc['_id']}")
    # print("Embedding:", doc["_source"]["embedding"])

Available indices: ['helpdesk-embeddings']
Num of embeddings in index: 15784
ID: 681691498
ID: 681841359
ID: 681859169
ID: 681995997
ID: 682210172
ID: 682251668
ID: 682255425
ID: 682558878
ID: 682672037
ID: 682682405


In [27]:
# delete the entire index after checking 
# ONLY RUN TO DELETE THE ENTIRE INDEX CREATED - SO THAT IT IS A CLEAN STATE IN VECTOR DB

# elasticsearch client 
es = Elasticsearch(
    hosts=["https://elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com"], 
    basic_auth=('elastic', '1rdzVXU8i8F9T8hs1p6c78d6'), 
    verify_certs=False
)

# es.indices.delete(index="helpdesk-embeddings", ignore_unavailable=True)
# print("Index deleted.\n")

# view available embedding indices 
indices = es.indices.get_alias().keys()
print("Available indices:", list(indices))

Available indices: ['helpdesk-embeddings']


<h1>Retrieving Top 3 Most Relevant Embeddings from Vector DB</h1>

In [28]:
# elasticsearch client 

import os

es_user = os.environ.get("elastic_user")
es_password = os.environ.get("elastic_password")


es = Elasticsearch(
    hosts=["https://elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com"], 
    basic_auth=(es_user, es_password), 
    verify_certs=False
)

mapping = es.indices.get_mapping(index="helpdesk-embeddings")
print(mapping["helpdesk-embeddings"]["mappings"]["properties"].get("embedding", "Field not found"))

{'type': 'dense_vector', 'dims': 384, 'index': True, 'similarity': 'cosine', 'index_options': {'type': 'int8_hnsw', 'm': 16, 'ef_construction': 100}}


In [29]:
# elasticsearch client 
es = Elasticsearch(
    hosts=["https://elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com"], 
    basic_auth=('elastic', '1rdzVXU8i8F9T8hs1p6c78d6'), 
    verify_certs=False
)

mapping = es.indices.get_mapping(index="helpdesk-embeddings")
print(mapping["helpdesk-embeddings"]["mappings"]["properties"].get("embedding", "Field not found"))

{'type': 'dense_vector', 'dims': 384, 'index': True, 'similarity': 'cosine', 'index_options': {'type': 'int8_hnsw', 'm': 16, 'ef_construction': 100}}


In [42]:
import warnings
import urllib3
from urllib3.exceptions import InsecureRequestWarning

# Disable SSL warnings
warnings.simplefilter('ignore', InsecureRequestWarning)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# elasticsearch client 
es = Elasticsearch(
    hosts=["https://elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com"], 
    basic_auth=('elastic', '1rdzVXU8i8F9T8hs1p6c78d6'), 
    verify_certs=False
)

model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device='cpu')


# get top 3 most relevant embeddings 
def search_embeddings(query, top_n=3):
    # Step 1: Embed the query using the same model
    query_embedding = model.encode(query).tolist()

    # Step 2: Search the Elasticsearch index for the closest vector matches
    response = es.search(
        index="helpdesk-embeddings",
        body={
            "size": top_n,
            "query": {
                "knn": {
                    "field": "embedding",
                    "k": top_n,
                    "num_candidates": 100,
                    "query_vector": query_embedding
                }
            }
        }
    )

    hits = response["hits"]["hits"]
    relevant_answers = []

    for hit in hits:
        issue_id = hit["_source"]["issue_id"]
        answer_body = hit["_source"]["answer_body"]
        score = hit["_score"]
        relevant_answers.append({
            "issue_id": issue_id,
            "answer_body": answer_body,
            "score": score
        })

    return relevant_answers

user_query = "Show me issue id 	issue_id 10060186"
top_matches = search_embeddings(user_query) 

for match in top_matches:
    print(f"Issue ID: {match['issue_id']}")
    print(f"Answer Body: {match['answer_body']}")
    print(f"Score: {match['score']}")
    print("="*50)
    

/opt/app-root/lib64/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:397: SecurityWarning: Connecting to 'https://elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(


Issue ID: 1386456451
Answer Body: This might be related to . > 👍 Hi . Which version of the Arduino IDE are you using? If you haven't already, please give the latest nightly build of Arduino IDE 2.x a try and then let us know whether the problem still occurs. The download links are listed here: <link> > Hi . Which version of the Arduino IDE are you using? > > If you haven't already, please give the latest nightly build of Arduino IDE 2.x a try and then let us know whether the problem still occurs. The download links are listed here: > > <link> Hello, I am using Arduino IDE 1.8.19, I'll try that version 2.0, thanks. (I've tried this 2.0 in the past, but still haven't had esp8266 OTA working via mDNS, I use [esp-link](<link> as UNO Wifi) Hi . Did you ever get the chance to give Arduino IDE 2.x a try? If so, did the problem still occur using that version? Hi, I tested the IDE 2.0 and this problem did not occur. I noticed a long delay for the IDE 2.0 to show the "About" window. Is there any

/tmp/ipykernel_2302/4199514187.py:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(


In [31]:
#Import required libraries
import requests
import pandas as pd
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch, helpers
from tqdm import tqdm
from elasticsearch.helpers import bulk
# model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device='cpu')
# model.save("multi-qa-MiniLM-L6-cos-v1")

<h1>Model Inference - Prompt Construction</h1>

In [37]:
import requests

#Query
user_query = "Can you teach me how to make an application using angular?"
# Step 1: Build the retrieved context
context = "\n\n---\n\n".join([
    f"Issue ID: {match['issue_id']}\nAnswer: {match['answer_body']}" for match in top_matches
])

# Step 2: Construct messages for /v1/chat/completions
messages = [
    {
        "role": "system",
        "content": (
            "You are a highly capable GitHub Helpdesk Support Assistant designed to assist users based on real GitHub issue threads.\n\n"
            "🔵 Carefully follow these specific instructions:\n\n"
            "1. Read the user question and the provided \"Context\" carefully.\n"
            "2. Think step-by-step using chain-of-thought reasoning:\n"
            "    - Summarize the user's question into clear problem keywords first.\n"
            "    - Identify whether relevant information exists in the provided Context.\n"
            "    - If sufficient context is found, proceed to answer the question, citing the related parts briefly.\n"
            "    - If no relevant context is found, politely say so and suggest the user refine or expand their question.\n"
            "3. Structure your response into these sections explicitly:\n"
            "    - **User Question Summary**\n"
            "    - **Context Match Analysis**\n"
            "    - **Final Answer**\n"
            "4. After each section output, STOP and ask the user:\n"
            "    - \"Would you like me to continue?\"\n"
            "5. Proceed only if the user explicitly agrees.\n\n"
            "🔵 Additional Strict Rules:\n"
            "- Never hallucinate answers if context is missing.\n"
            "- If unsure, always say so politely.\n"
            "- Pause after each major step for user feedback.\n\n"
            "---\n\n"
            f"Context:\n{context}"
        )
    },
    {
        "role": "user",
        "content": user_query
    }
]

# Step 3: Construct payload
payload = {
    "model": "model",  # Replace with your actual model ID
    "messages": messages,
    "max_tokens": 1024,
    "temperature": 0.3,
    "top_p": 1,
    "n": 1,
    "repetition_penalty": 1.1,
    "presence_penalty": 0.2,
    "frequency_penalty": 0.2,
    "stream": False
}

# Step 4: Query the LLM server
response = requests.post(infer_url, json=payload)

# Step 5: Process and print output
output_body = response.json()
generated_response = output_body['choices'][0]['message']['content']
print(generated_response.strip())

# Step 6: Optionally append the generated response back into the conversation history
messages.append({"role": "system", "content": generated_response.strip()})

🔵 User Question Summary:
The user is asking for guidance on creating an application using Angular.

🔵 Context Match Analysis:
The provided context does not contain any information directly related to creating applications with Angular. It consists of discussions about Pelican, a static site generator, and its related issues such as licensing notices, parser errors, and locale settings.

🔵 Final Answer:
I'm sorry for the inconvenience, but the given context doesn't cover tutorials or guides on building applications with Angular. However, I can provide you with a basic outline to start learning Angular development:

1. **Set up your environment**: Install Node.js and npm (Node Package Manager) if you haven't already. Then, install Angular CLI globally via npm (`npm install -g @angular/cli`).

2. **Create a new project**: Use the Angular CLI to generate a new project (`ng new my-app`), where 'my-app' is the name of your application. Navigate into your new project directory (`cd my-app`).


## Version 1

In [ ]:
# import requests
# import re
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# from sentence_transformers import SentenceTransformer

# # Step 0: Initialize
# messages = [
#     {
#         "role": "system",
#         "content": (
#             "You are a highly capable GitHub Helpdesk Support Assistant designed to assist users based on real GitHub issue threads.\n\n"
#             "Always maintain a helpful, professional tone.\n\n"
#             "Carefully read the user's question.\n\n"
#             "Use context if available.\n\n"
#             "Only suggest clarifying questions or general troubleshooting advice if specifically instructed."
#         )
#     }
# ]

# model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device='cpu')

# # Step 1: Helper functions
# def build_context(top_matches):
#     return "\n\n---\n\n".join([
#         f"{match['answer_body']}" for match in top_matches
#     ])

# def summarize_old_messages(messages, preserve_n=20):
#     if len(messages) <= preserve_n:
#         return messages
    
#     old_conversation = []
#     for msg in messages[1:-preserve_n]:
#         if msg['role'] in ['user', 'assistant']:
#             old_conversation.append(f"{msg['role'].capitalize()}: {msg['content']}")
    
#     summary = "🔵 Summary of earlier conversation:\n" + "\n".join(
#         f"- {line}" for line in old_conversation
#     )
    
#     summarized_messages = [
#         messages[0],
#         {"role": "system", "content": summary}
#     ] + messages[-preserve_n:]
    
#     return summarized_messages

# def collapse_conversation_memory(messages):
#     conversation_summary = []
#     for msg in messages[1:]:
#         if msg['role'] in ['user', 'assistant']:
#             conversation_summary.append(f"{msg['role'].capitalize()}: {msg['content']}")
    
#     final_summary = "🔵 Resolved Issue Summary:\n" + "\n".join(
#         f"- {line}" for line in conversation_summary
#     )
    
#     collapsed_messages = [
#         messages[0],
#         {"role": "system", "content": final_summary}
#     ]
    
#     return collapsed_messages

# def get_embedding(text):
#     return model.encode(text)

# def calculate_similarity(embedding1, embedding2):
#     embedding1 = np.array(embedding1).reshape(1, -1)
#     embedding2 = np.array(embedding2).reshape(1, -1)
#     return cosine_similarity(embedding1, embedding2)[0][0]

# # Step 2: State Variables
# initial_topic_embedding = None
# context_injected = False
# guiding_questions_done = False
# clarification_rounds = 0

# # Step 3: Core function
# def send_message(user_query):
#     global messages
#     global initial_topic_embedding
#     global context_injected
#     global guiding_questions_done
#     global clarification_rounds

#     user_embedding = get_embedding(user_query)

#     # Detect topic change only if input is substantial
#     if initial_topic_embedding is None:
#         initial_topic_embedding = user_embedding
#     elif len(user_query.split()) > 5:  # Ignore very short replies like "it", "1.8.19"
#         similarity = calculate_similarity(initial_topic_embedding, user_embedding)
#         if similarity < 0.5:
#             print(f"🔄 Major topic change detected (similarity {similarity:.2f}). Retriggering RAG...")
#             context_injected = False
#             initial_topic_embedding = user_embedding
#             guiding_questions_done = False
#             clarification_rounds = 0

#     if not context_injected:
#         top_matches = search_embeddings(user_query)
#         min_score_threshold = 0.4

#         if not top_matches or top_matches[0]['score'] < min_score_threshold:
#             context_message = {
#                 "role": "system",
#                 "content": (
#                     "🔵 Context Update:\n\n"
#                     "No strong matching past issues found.\n\n"
#                     "(Answer politely based on general knowledge.)"
#                 )
#             }
#         else:
#             context = build_context(top_matches)
#             quoted_context = context.replace('\n', '\n> ')
#             context_message = {
#                 "role": "system",
#                 "content": (
#                     "🔵 Context Update:\n\n"
#                     "Summaries of past similar issues (use carefully):\n\n"
#                     f"> {quoted_context}\n\n"
#                     "(Only use if truly matching.)"
#                 )
#             }
        
#         messages.append(context_message)
#         context_injected = True

#     # Step 2: Behavior Control
#     user_text_lower = user_query.lower()
#     response_behavior = "normal"

#     vague_keywords = ["problem", "idk", "not sure", "nothing working"]

#     if any(vague in user_text_lower for vague in vague_keywords):
#         if not guiding_questions_done:
#             response_behavior = "guiding_questions"
#             guiding_questions_done = True
#         else:
#             clarification_rounds += 1
#             if clarification_rounds >= 2:
#                 response_behavior = "general_causes"
#             else:
#                 response_behavior = "follow_up_question"

#     # Step 3: Insert dynamic system message to guide LLM
#     if response_behavior == "guiding_questions":
#         behavior_instruction = {
#             "role": "system",
#             "content": (
#                 "🔵 Special Behavior Instruction:\n"
#                 "The user seems unsure. Kindly ask around 5 short guiding questions to clarify their issue."
#             )
#         }
#     elif response_behavior == "follow_up_question":
#         behavior_instruction = {
#             "role": "system",
#             "content": (
#                 "🔵 Special Behavior Instruction:\n"
#                 "The user provided partial information. Ask 1 short follow-up question to get more detail."
#             )
#         }
#     elif response_behavior == "general_causes":
#         behavior_instruction = {
#             "role": "system",
#             "content": (
#                 "🔵 Special Behavior Instruction:\n"
#                 "The user remains unclear after multiple tries. Politely suggest some general possible causes based on common troubleshooting experience."
#             )
#         }
#     else:
#         behavior_instruction = None

#     if behavior_instruction:
#         messages.append(behavior_instruction)

#     # Step 4: Add user query
#     messages.append({"role": "user", "content": user_query})

#     # Step 5: Build payload
#     payload = {
#         "model": "model",  # <-- Keep "model" exactly as you requested
#         "messages": messages,
#         "max_tokens": 512,
#         "temperature": 0.3,
#         "top_p": 1,
#         "n": 1,
#         "repetition_penalty": 1.1,
#         "presence_penalty": 0.2,
#         "frequency_penalty": 0.2,
#         "stream": False
#     }

#     response = requests.post(infer_url, json=payload)
    
#     if response.status_code == 200:
#         output_body = response.json()
#         generated_response = output_body['choices'][0]['message']['content']
        
#         tokens_generated = len(generated_response.split())
#         if tokens_generated > 300:
#             generated_response += "\n\nWould you like me to continue?"

#         messages.append({"role": "assistant", "content": generated_response.strip()})

#         if len(messages) > 30:
#             messages = summarize_old_messages(messages, preserve_n=15)
        
#         return generated_response.strip()
#     else:
#         return f"⚠️ Error {response.status_code}: {response.text}"

# # Step 4: User interaction loop
# print("🗨️ Welcome to GitHub Helpdesk Chatbot!")
# print("Type 'exit' to end the conversation.\n")

# while True:
#     user_query = input("👤 You: ")
#     if user_query.lower() in ["exit", "quit"]:
#         print("👋 Ending the chat. Goodbye!")
#         break
    
#     if any(keyword in user_query.lower() for keyword in ["solved", "fixed", "thank you", "thanks", "resolved", "issue resolved"]):
#         print("✅ Issue marked as solved! Collapsing conversation memory...")
#         messages = collapse_conversation_memory(messages)
#         continue
    
#     bot_reply = send_message(user_query)
#     print(f"🤖 Assistant: {bot_reply}\n")


## Version 2

In [65]:
import requests
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Step 0: Initialize
messages = [
    {
        "role": "system",
        "content": (
            "You are a highly capable GitHub Helpdesk Support Assistant designed to assist users based on real GitHub issue threads.\n\n"
            "Always maintain a professional and helpful tone.\n\n"
            "Carefully read the user's question and the context provided.\n\n"
            "Use context if available. Only refer to it when relevant, and never make assumptions about the user's problem.\n\n"
            "You will guide the user through the troubleshooting process in a helpful, friendly manner.\n\n"
            "🔵 Special Instructions:\n\n"
            "1. If the user provides vague input, ask guiding questions (max 5 questions).\n"
            "2. If clarification is still needed, follow up with 1 small question.\n"
            "3. After 2 failed clarification attempts, suggest general causes based on past insights.\n\n"
            "🔵 When responding:\n"
            "    - If the user remains unclear after clarification, suggest **General Causes**.\n"
            "    - If the user provides partial information, ask **1 short follow-up question**.\n"
            "    - Otherwise, keep the response brief and professional.\n\n"
            "🔵 Additional Notes:\n"
            "- Maintain a friendly, professional, and clear tone.\n"
            "- Only ask 'Would you like me to continue?' if the response is longer than 300 words.\n"
            "- Ensure all responses are helpful and directed towards resolving the issue without making unnecessary assumptions.\n\n"
            "🔵 Be respectful and polite, even when suggesting possible general causes or follow-ups."
        )
    }
]

model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device='cpu')

# Step 1: Helper functions
def build_context(top_matches):
    return "\n\n---\n\n".join([
        f"{match['answer_body']}" for match in top_matches
    ])

def summarize_old_messages(messages, preserve_n=20):
    if len(messages) <= preserve_n:
        return messages
    
    old_conversation = []
    for msg in messages[1:-preserve_n]:
        if msg['role'] in ['user', 'assistant']:
            old_conversation.append(f"{msg['role'].capitalize()}: {msg['content']}")
    
    summary = "🔵 Summary of earlier conversation:\n" + "\n".join(
        f"- {line}" for line in old_conversation
    )
    
    summarized_messages = [
        messages[0],
        {"role": "system", "content": summary}
    ] + messages[-preserve_n:]
    
    return summarized_messages

def collapse_conversation_memory(messages):
    conversation_summary = []
    for msg in messages[1:]:
        if msg['role'] in ['user', 'assistant']:
            conversation_summary.append(f"{msg['role'].capitalize()}: {msg['content']}")
    
    final_summary = "🔵 Resolved Issue Summary:\n" + "\n".join(
        f"- {line}" for line in conversation_summary
    )
    
    collapsed_messages = [
        messages[0],
        {"role": "system", "content": final_summary}
    ]
    
    return collapsed_messages

def get_embedding(text):
    return model.encode(text)

def calculate_similarity(embedding1, embedding2):
    embedding1 = np.array(embedding1).reshape(1, -1)
    embedding2 = np.array(embedding2).reshape(1, -1)
    return cosine_similarity(embedding1, embedding2)[0][0]

# Step 2: State Variables
initial_topic_embedding = None
context_injected = False
guiding_questions_done = False
clarification_rounds = 0

# Step 3: Core function
def send_message(user_query):
    global messages
    global initial_topic_embedding
    global context_injected
    global guiding_questions_done
    global clarification_rounds

    user_embedding = get_embedding(user_query)

    # Detect topic change only if input is substantial
    if initial_topic_embedding is None:
        initial_topic_embedding = user_embedding
    elif len(user_query.split()) > 5:  # Ignore very short replies like "it", "1.8.19"
        similarity = calculate_similarity(initial_topic_embedding, user_embedding)
        if similarity < 0.5:
            print(f"🔄 Major topic change detected (similarity {similarity:.2f}). Retriggering RAG...")
            context_injected = False
            initial_topic_embedding = user_embedding
            guiding_questions_done = False
            clarification_rounds = 0

    if not context_injected:
        top_matches = search_embeddings(user_query)
        min_score_threshold = 0.4

        if not top_matches or top_matches[0]['score'] < min_score_threshold:
            context_message = {
                "role": "system",
                "content": (
                    "🔵 Context Update:\n\n"
                    "No strong matching past issues found.\n\n"
                    "(Answer politely based on general knowledge.)"
                )
            }
        else:
            context = build_context(top_matches)
            quoted_context = context.replace('\n', '\n> ')
            context_message = {
                "role": "system",
                "content": (
                    "🔵 Context Update:\n\n"
                    "Summaries of past similar issues (use carefully):\n\n"
                    f"> {quoted_context}\n\n"
                    "(Only use if truly matching.)"
                )
            }
        
        messages.append(context_message)
        context_injected = True

    # Step 2: Behavior Control
    user_text_lower = user_query.lower()
    response_behavior = "normal"

    vague_keywords = ["problem", "idk", "not sure", "nothing working"]

    if any(vague in user_text_lower for vague in vague_keywords):
        if not guiding_questions_done:
            response_behavior = "guiding_questions"
            guiding_questions_done = True
        else:
            clarification_rounds += 1
            if clarification_rounds >= 2:
                response_behavior = "general_causes"
            else:
                response_behavior = "follow_up_question"

    # Step 3: Insert dynamic system message to guide LLM
    if response_behavior == "guiding_questions":
        behavior_instruction = {
            "role": "system",
            "content": (
                "🔵 Special Behavior Instruction:\n"
                "The user seems unsure. Kindly ask around 5 short guiding questions to clarify their issue."
            )
        }
    elif response_behavior == "follow_up_question":
        behavior_instruction = {
            "role": "system",
            "content": (
                "🔵 Special Behavior Instruction:\n"
                "The user provided partial information. Ask 1 short follow-up question to get more detail."
            )
        }
    elif response_behavior == "general_causes":
        behavior_instruction = {
            "role": "system",
            "content": (
                "🔵 Special Behavior Instruction:\n"
                "The user remains unclear after multiple tries. Politely suggest some general possible causes based on common troubleshooting experience."
            )
        }
    else:
        behavior_instruction = None

    if behavior_instruction:
        messages.append(behavior_instruction)

    # Step 4: Add user query
    messages.append({"role": "user", "content": user_query})

    # Step 5: Build payload
    payload = {
        "model": "model",  # <-- Keep "model" exactly as you requested
        "messages": messages,
        "max_tokens": 512,
        "temperature": 0.3,
        "top_p": 1,
        "n": 1,
        "repetition_penalty": 1.1,
        "presence_penalty": 0.2,
        "frequency_penalty": 0.2,
        "stream": False
    }

    response = requests.post(infer_url, json=payload)
    
    if response.status_code == 200:
        output_body = response.json()
        generated_response = output_body['choices'][0]['message']['content']
        
        tokens_generated = len(generated_response.split())
        if tokens_generated > 300:
            generated_response += "\n\nWould you like me to continue?"

        messages.append({"role": "assistant", "content": generated_response.strip()})

        if len(messages) > 30:
            messages = summarize_old_messages(messages, preserve_n=15)
        
        return generated_response.strip()
    else:
        return f"⚠️ Error {response.status_code}: {response.text}"

# Step 4: User interaction loop
print("🗨️ Welcome to GitHub Helpdesk Chatbot!")
print("Type 'exit' to end the conversation.\n")

while True:
    user_query = input("👤 You: ")
    if user_query.lower() in ["exit", "quit"]:
        print("👋 Ending the chat. Goodbye!")
        break
    
    if any(keyword in user_query.lower() for keyword in ["solved", "fixed", "thank you", "thanks", "resolved", "issue resolved"]):
        print("✅ Issue marked as solved! Collapsing conversation memory...")
        messages = collapse_conversation_memory(messages)
        continue
    
    bot_reply = send_message(user_query)
    print(f"🤖 Assistant: {bot_reply}\n")


🗨️ Welcome to GitHub Helpdesk Chatbot!
Type 'exit' to end the conversation.



👤 You:  hello bitch


/tmp/ipykernel_2302/4199514187.py:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(


🤖 Assistant: 🔵 I'm here to help! However, I'd appreciate it if you could rephrase your request in a more constructive manner so I can better understand and assist you with your GitHub issue. Could you please tell me what specific problem you're encountering? Thank you.



👤 You:  I have an issue with my arduino ide


...

🤖 Assistant: 🔵 I see. This issue might be related to the Arduino IDE trying to access a serial port that's no longer connected when you unplug your device. Here are a few steps to troubleshoot:

1. Check if the correct board and port are selected in your Arduino IDE under Tools > Board and Tools > Port.
2. Make sure you've properly closed any sketches before disconnecting your board.
3. Try updating your Arduino IDE to the latest stable version (1.8.19 at the moment).
4. Disable the "Verify Code Before Upload" option in File > Preferences > Verify Before Upload, then enable it again.
5. If none of these work, consider uninstalling and reinstalling the Arduino IDE, ensuring you delete any leftover files or folders associated with previous installations.

Please let me know how it goes or if you need further assistance!



👤 You:  It's solved


✅ Issue marked as solved! Collapsing conversation memory...


👤 You:  exit


👋 Ending the chat. Goodbye!


<h1>Request Function</h1>

Build and submit the REST request. 

Note: You submit the data in the same format that you used for an ONNX inference.

In [2]:
import requests


def rest_request(prompt):
    json_data = {
        "model": "llm",
        "prompt": [
            prompt
        ],
        "max_tokens": 512,
        "temperature": 1,
        "top_p": 1,
        "n": 1,
        "stream": False,
        "logprobs": 0,
        "echo": False,
        "stop": [
            "string"
        ],
        "presence_penalty": 0,
        "frequency_penalty": 0,
        "best_of": 1,
        "user": "string",
        "top_k": -1,
        "ignore_eos": False,
        "use_beam_search": False,
        "stop_token_ids": [
            0
        ],
        "skip_special_tokens": True,
        "spaces_between_special_tokens": True,
        "repetition_penalty": 1,
        "min_p": 0,
        "include_stop_str_in_output": False,
        "length_penalty": 1
    }

    response = requests.post(infer_url, json=json_data, verify=False)
    return response.json()

In [3]:
prediction = rest_request("What accelerators are supported in openshift AI?")
prediction

ConnectionError: HTTPConnectionPool(host='model-predictor.minio.svc.cluster.local', port=80): Max retries exceeded with url: /v1/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fadbd8a11d0>: Failed to establish a new connection: [Errno 111] Connection refused'))